
### Learn how to  use NLTK

In [1]:
# load required lbraries
import pandas as pd
import numpy as np
import nltk

In [2]:
# making width of the column viewable
pd.set_option('display.max_colwidth', None)

In [3]:
# load data
data = pd.read_csv('data/imdb_labelled.csv')
data.head()

,text,label
0,"A very, very, very slow-moving, aimless movie about a distressed, drifting young man.",0
1,"Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.",0
2,"Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.",0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.,1


In [4]:
data['label'].value_counts()

1    386
0    362
Name: label, dtype: int64

In [5]:
sample = data.text[0]
sample

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '

### Tokens and Bigrams

In [6]:
# import package
from nltk import word_tokenize

# tokenize the sample
sample_tokens = word_tokenize(sample)
sample_tokens[:10]

['A', 'very', ',', 'very', ',', 'very', 'slow-moving', ',', 'aimless', 'movie']

### Creating a list of Bigrams

In [7]:
from nltk import bigrams

# bigrams of the sample
sample_bigragrams = list(bigrams(sample_tokens))
sample_bigragrams[:10]

[('A', 'very'),
 ('very', ','),
 (',', 'very'),
 ('very', ','),
 (',', 'very'),
 ('very', 'slow-moving'),
 ('slow-moving', ','),
 (',', 'aimless'),
 ('aimless', 'movie'),
 ('movie', 'about')]

### Frequency Distribution

In [8]:
from nltk import FreqDist

sample_fd = FreqDist(sample_tokens)
sample_fd.most_common(10)

[(',', 4),
 ('very', 3),
 ('A', 1),
 ('slow-moving', 1),
 ('aimless', 1),
 ('movie', 1),
 ('about', 1),
 ('a', 1),
 ('distressed', 1),
 ('drifting', 1)]

In [9]:
# create a function to accept a text and n nad returns top most common tokens
def text_n(text, n):
    # get text tokens/unigrams     
    tokenize = word_tokenize(text)    
    # get frequency distribution for the unigrams
    fd = FreqDist(tokenize)
    # return the top n most common unigrams
    return fd.most_common(n)

# try sample
text_n(data.text[1], 10)

[('the', 2),
 ('Not', 1),
 ('sure', 1),
 ('who', 1),
 ('was', 1),
 ('more', 1),
 ('lost', 1),
 ('-', 1),
 ('flat', 1),
 ('characters', 1)]

### Document Term Matrix (DTM)

In [10]:
# rep the frequency of terms that occur in a collection of documents
# create a dtm function 
# import package
from sklearn.feature_extraction.text import CountVectorizer

def create_dtm(series):
    # create an instance of countvectorizer
    cv = CountVectorizer()
    # create a dtm from provided series
    dtm = cv.fit_transform(series)
    # convert sparse array to dense array
    dtm = dtm.todense()
    # get column names
    features = cv.get_feature_names_out()
    # create df
    dtm_df = pd.DataFrame(dtm, columns=features)
    
    return dtm_df
# try on sample with 5 rows
create_dtm(data.text.head())

,about,acting,aimless,almost,and,angles,anything,artiness,as,attempting,...,trying,very,walked,was,when,white,who,whom,with,young
0,1,0,1,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,1,0,0
2,0,1,0,1,3,1,0,1,1,1,...,0,0,0,1,0,1,0,0,1,0
3,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0


### Feature Importance

In [11]:
# define a function named top_n_tokens
# import logistic regression
from sklearn.linear_model import LogisticRegression

def top_n_tokens(text, sentiment, n):
    # create instance of the class
    lrg = LogisticRegression(solver = 'lbfgs', max_iter = 2500, random_state = 1234)
    cv = CountVectorizer()
    # create dtm
    dtm = cv.fit_transform(text)
    # fit logistic regression model
    lrg.fit(dtm , sentiment)
    # get the coefficient
    coefs = lrg.coef_[0]
    # create the feature / column names
    features = cv.get_feature_names_out()
    # create a dataframe
    df = pd.DataFrame({"Tokens": features, "Coefficients": coefs})
    
    # return the largest n
    return df.nlargest(n, 'Coefficients')


# try with sample
top_n_tokens(data.text, data.label, 10)

,Tokens,Coefficients
1567,liked,1.286747
2997,wonderful,1.242158
1104,funny,1.112821
1182,great,1.068772
2949,well,1.043139
246,beautiful,1.042833
0,10,1.035405
344,brilliant,1.014080
908,excellent,1.009914
2203,right,0.985806


In [12]:
# smallest coefficeint
def top_n_tokens(text, sentiment, n):
    # create instance of the class
    lrg = LogisticRegression(solver = 'lbfgs', max_iter = 2500, random_state = 1234)
    cv = CountVectorizer()
    # create dtm
    dtm = cv.fit_transform(text)
    # fit logistic regression model
    lrg.fit(dtm , sentiment)
    # get the coefficient
    coefs = lrg.coef_[0]
    # create the feature / column names
    features = cv.get_feature_names_out()
    # create a dataframe
    df = pd.DataFrame({"Tokens": features, "Coefficients": coefs})
    
    # return the largest n
    return df.nsmallest(n, 'Coefficients')


# try with sample
top_n_tokens(data.text, data.label, 10)

,Tokens,Coefficients
222,bad,-1.872751
211,awful,-1.334554
2530,stupid,-1.175416
441,cheap,-1.139512
1802,no,-1.137234
893,even,-1.091436
3017,would,-1.047931
3012,worst,-1.039231
2923,waste,-1.038206
1819,nothing,-0.973472


### Pre-trained ---- TextBlob

In [15]:
# define function called polarity_subjectivity
# import Textblob
from textblob import TextBlob

def polarity_subjectivity(text=sample, print_results=False):
    # create an instance of Textblob
    tb = TextBlob(text)
    # if condition is met print results otherwise return the tuple
    if print_results:
        print(f"Polarity is {round(tb.sentiment[0], 2)} and Subjectivity is {round(tb.sentiment[1], 2)}.")
    else:
        return (tb.sentiment[0], tb.sentiment[1])

# try with sample
polarity_subjectivity(sample, print_results=True)

Polarity is 0.18 and Subjectivity is 0.4.


In [16]:
# solution 6
# load library

from nltk import word_tokenize

def token_count(string):
    # number of tokens in a string
    return len(word_tokenize(string))


def series_tokens(series):
    # apply token_count function on pandas series
    return series.apply(token_count)

# apply function
series_tokens(data.text.head(10))

0    18
1    21
2    33
3     9
4    22
5    27
6     4
7    17
8     4
9    11
Name: text, dtype: int64

In [17]:
# series polarity subjectivity
def series_polarity_subjectivity(series):
    return series.apply(polarity_subjectivity)

# apply function
series_polarity_subjectivity(data.text.head(10))

0                                 (0.18, 0.395)
1    (0.014583333333333337, 0.4201388888888889)
2    (-0.12291666666666666, 0.5145833333333333)
3                  (-0.24375000000000002, 0.65)
4                                    (1.0, 0.3)
5                                   (-0.1, 0.5)
6                                   (-0.2, 0.0)
7                     (0.7, 0.6000000000000001)
8                                   (-0.2, 0.5)
9                                    (0.7, 0.8)
Name: text, dtype: object

### Measure of Complexity and Lexical Diversity

In [36]:
def complexity(string):
    # create a list of all tokens
    tokens = word_tokenize(string)
    
    # create a set oof all tokens
    unique_tokens = set(tokens)
    
    #return complexity measure
    try:        
        cm =len(unique_tokens)/ len(tokens)
    except ZeroDivisionError:
        cm = 0
    return cm

# try function
data.text.head(10).apply(complexity)

0    0.722222
1    0.952381
2    0.848485
3    1.000000
4    1.000000
5    0.814815
6    1.000000
7    0.941176
8    1.000000
9    0.909091
Name: text, dtype: float64

### Text Cleanup

In [37]:
# import library
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')

english_stop_words[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [38]:
# stopword remover
def stopword_remover(string):
    # tokenize the string
    tokens = word_tokenize(string)
    # create a list of english stopwords
    english_stopwords = stopwords.words('english')
    # return non stopwrds
    return [ w for w in tokens if w.lower() not in english_stopwords]

# apply function
data.text.head().apply(stopword_remover)

0                                                                                                 [,, ,, slow-moving, ,, aimless, movie, distressed, ,, drifting, young, man, .]
1                                                                                                        [sure, lost, -, flat, characters, audience, ,, nearly, half, walked, .]
2    [Attempting, artiness, black, &, white, clever, camera, angles, ,, movie, disappointed, -, became, even, ridiculous, -, acting, poor, plot, lines, almost, non-existent, .]
3                                                                                                                                            [little, music, anything, speak, .]
4                                                                                                     [best, scene, movie, Gerardo, trying, find, song, keeps, running, head, .]
Name: text, dtype: object

In [39]:
# nonalpha remover
def stopword_remover_nonalpha(string):
    return [ w for w in stopword_remover(string) if w.isalpha()]
# apply function
data.text.head().apply(stopword_remover_nonalpha)

0                                                                                                [aimless, movie, distressed, drifting, young, man]
1                                                                                    [sure, lost, flat, characters, audience, nearly, half, walked]
2    [Attempting, artiness, black, white, clever, camera, angles, movie, disappointed, became, even, ridiculous, acting, poor, plot, lines, almost]
3                                                                                                                  [little, music, anything, speak]
4                                                                           [best, scene, movie, Gerardo, trying, find, song, keeps, running, head]
Name: text, dtype: object

In [41]:
# complexity cleaned up

def complexity_cleaned(series):
    return series.apply(lambda x: complexity(" ".join(stopword_remover_nonalpha(x))))
# add complexity cleaned to df
data['Complexity'] = complexity_cleaned(data.text)
data.sort_values('Complexity', ascending=False).head(10)

,text,label,Complexity
0,"A very, very, very slow-moving, aimless movie about a distressed, drifting young man.",0,1.0
484,Kris Kristoffersen is good in this movie and really makes a difference.,1,1.0
476,Tom Wilkinson broke my heart at the end... and everyone else's judging by the amount of fumbling for hankies and hands going up to faces among males and females alike.,1,1.0
477,Julian Fellowes has triumphed again.,1,1.0
478,He's a national treasure.,1,1.0
479,GO AND SEE IT!,1,1.0
480,This is an excellent film.,1,1.0
481,The aerial scenes were well-done.,1,1.0
482,It was also the right balance of war and love.,1,1.0
483,"The film gives meaning to the phrase, ""Never in the history of human conflict has so much been owed by so many to so few.",1,1.0
